In [1]:
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [2]:
import numpy as np
import pinocchio as pin
import time
import os
import re
from pinocchio.robot_wrapper import RobotWrapper

from utils import *
from inverse_kinematics import *

In [3]:
%%capture
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess()
viewer = meshcat.Visualizer(zmq_url=zmq_url)

ip_addr_regex = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
public_ip = '137.184.48.127'
public_url = re.sub(ip_addr_regex, public_ip, web_url)
viewer.window.web_url = public_url

In [14]:
robot = RobotWrapper.BuildFromURDF('./urdf/arm.urdf', './urdf')
viz = pin.visualize.MeshcatVisualizer(
    robot.model, robot.collision_model, robot.visual_model
)
viz.initViewer(viewer)
viz.loadViewerModel()
vis = viz.viewer
print('You can open the visualizer by visiting the following URL:')
print(viewer.window.web_url)
viz.display(np.zeros(5))
vis["wrist"].set_transform(tf.translation_matrix([0.1,0,-0.7]))
vis["elbow"].set_transform(tf.translation_matrix([0.1,0,-0.35]))

You can open the visualizer by visiting the following URL:
http://137.184.48.127:7001/static/


In [5]:
config = {'vertical': ['Top', 'Middle','Bottom'],
          'horizontal': ['Left', 'Center', 'Right'],
          'traj_ids': [1, 2],
          'subject_ids': [2, 3, 4]}
traj = getExtendedTraj(0, 0, "PositionTopLeftTrajectory1Subject3", config)

In [6]:
wrist_pos = traj[:, 17:20]
elbow_pos = traj[:, 5:8]
shoulder_pos = traj[:, 11:14]
wrist_rel = 1 / 1000 * (wrist_pos - shoulder_pos)
elbow_rel = 1 / 1000 * (elbow_pos - shoulder_pos)

In [7]:
def calculateJointTrajWrist(traj):
    wrist_pos = traj[:, 17:20]
    shoulder_pos = traj[:, 11:14]
    wrist_rel = 1 / 1000 * (wrist_pos - shoulder_pos)
    q = np.zeros(5)
    traj_q = np.zeros((len(wrist_rel), 5))
    frame_id = robot.model.getFrameId('wrist')
    q_previous = findAngle(wrist_rel[0, :], q, robot, frame_id)          

    for n in range(len(wrist_rel)):
        p_des = wrist_rel[n, :]
        q = findAngle(p_des, q, robot, frame_id)
        
        if isinstance(q, bool):
            q = q_previous
        else: 
            q_previous = findAngle(p_des, q, robot, frame_id)          
            
        traj_q[n, :] = q_previous
    
    return traj_q

In [8]:
def calculateJointTrajWristElbow(traj):
    wrist_pos = traj[:, 17:20]
    elbow_pos = traj[:, 5:8]
    shoulder_pos = traj[:, 11:14]
    wrist_rel = 1 / 1000 * (wrist_pos - shoulder_pos)
    elbow_rel = 1 / 1000 * (elbow_pos - shoulder_pos)
    
    traj_q = np.zeros((len(wrist_rel), 5))
    frame_id_wrist = robot.model.getFrameId('wrist')
    frame_id_elbow = robot.model.getFrameId('elbow')
    angles_previous = findAngleTwoFrames(robot, wrist_rel[0, :], elbow_rel[0, :], 
                                         frame_id_wrist, frame_id_elbow, np.zeros(5))
    
    for n in range(len(wrist_rel)):
        p_des_wrist = wrist_rel[n, :]
        p_des_elbow = elbow_rel[n, :]
        angles = findAngleTwoFrames(robot, p_des_wrist, p_des_elbow, 
                                    frame_id_wrist, frame_id_elbow, angles_previous)
        
        if isinstance(angles, bool):
            angles = angles_previous
        else: 
            angles_previous = findAngleTwoFrames(robot, p_des_wrist, p_des_elbow, 
                                                 frame_id_wrist, frame_id_elbow, angles_previous)          
            
        traj_q[n, :] = angles


    return traj_q

In [9]:
traj_q_wrist = calculateJointTrajWrist(traj)
traj_q_wrist_elbow = calculateJointTrajWristElbow(traj)

In [27]:
traj_viz = traj_q_wrist_elbow
vis["elbow"].set_object(g.Sphere(1/40), 
                        g.MeshLambertMaterial(
                             color=0x0000FF,
                             reflectivity=0))
vis["wrist"].set_object(g.Sphere(1/40), 
                        g.MeshLambertMaterial(
                             color=0xFF0000,
                             reflectivity=0))

for n in range(len(traj_viz)):
    vis["wrist"].set_transform(tf.translation_matrix(wrist_rel[n]))
    vis["elbow"].set_transform(tf.translation_matrix(elbow_rel[n]))
    viz.display(traj_viz[n])
    time.sleep(0.00125)

In [35]:
traj_viz = traj_q_wrist_elbow
n = 400
vis["wrist"].set_transform(tf.translation_matrix(wrist_rel[n]))
vis["elbow"].set_transform(tf.translation_matrix(elbow_rel[n]))
viz.display(traj_viz[n])
time.sleep(0.00125)

In [ ]:
from meshcat.animation import Animation
import meshcat.transformations as tf
from meshcat.animation import convert_frames_to_video

In [ ]:
anim = Animation()

camera_path = "/Cameras/default/rotated/<object>"

with anim.at_frame(vis, 0) as frame:
    frame[camera_path].set_property("zoom", "number", 1)
with anim.at_frame(vis, 30) as frame:
    frame[camera_path].set_property("zoom", "number", 0.5)
with anim.at_frame(vis, 60) as frame:
    frame[camera_path].set_property("zoom", "number", 1)
    
# While we're animating the camera zoom, we can also animate any other
# properties we want. Let's simultaneously translate the box during 
# the same animation:
with anim.at_frame(vis, 0) as frame:
    frame["box1"].set_transform(tf.translation_matrix([0, -1, 0]))
with anim.at_frame(vis, 60) as frame:
    frame["box1"].set_transform(tf.translation_matrix([0, 1, 0]))

vis.set_animation(anim)

In [ ]:
convert_frames_to_video("test_animation.tar", overwrite=True)